In [2]:
import numpy as np
import scipy as sp

In [3]:
import matplotlib.pylab as plt

In [5]:
import pyaudio, wave, sys

In [7]:
class AudioSource(object):
    def __init__(self):
        raise NotImplementedError("this is an abstract class")

    def __enter__(self):
        raise NotImplementedError("this is an abstract class")

    def __exit__(self, exc_type, exc_value, traceback):
        raise NotImplementedError("this is an abstract class")

In [8]:
class Microphone(AudioSource):
    """
    This is available if PyAudio is available, and is undefined otherwise.
    Creates a new ``Microphone`` instance, which represents a physical microphone on the computer. Subclass of ``AudioSource``.
    If ``device_index`` is unspecified or ``None``, the default microphone is used as the audio source. Otherwise, ``device_index`` should be the index of the device to use for audio input.
    A device index is an integer between 0 and ``pyaudio.get_device_count() - 1`` (assume we have used ``import pyaudio`` beforehand) inclusive. It represents an audio device such as a microphone or speaker. See the `PyAudio documentation <http://people.csail.mit.edu/hubert/pyaudio/docs/>`__ for more details.
    The microphone audio is recorded in chunks of ``chunk_size`` samples, at a rate of ``sample_rate`` samples per second (Hertz).
    Higher ``sample_rate`` values result in better audio quality, but also more bandwidth (and therefore, slower recognition). Additionally, some machines, such as some Raspberry Pi models, can't keep up if this value is too high.
    Higher ``chunk_size`` values help avoid triggering on rapidly changing ambient noise, but also makes detection less sensitive. This value, generally, should be left at its default.
    """
    def __init__(self, device_index = None, sample_rate = 16000, chunk_size = 1024):
        assert device_index is None or isinstance(device_index, int), "Device index must be None or an integer"
        if device_index is not None: # ensure device index is in range
            audio = pyaudio.PyAudio(); count = audio.get_device_count(); audio.terminate() # obtain device count
            assert 0 <= device_index < count, "Device index out of range"
        assert isinstance(sample_rate, int) and sample_rate > 0, "Sample rate must be a positive integer"
        assert isinstance(chunk_size, int) and chunk_size > 0, "Chunk size must be a positive integer"
        self.device_index = device_index
        self.format = pyaudio.paInt16 # 16-bit int sampling
        self.SAMPLE_WIDTH = pyaudio.get_sample_size(self.format) # size of each sample
        self.SAMPLE_RATE = sample_rate # sampling rate in Hertz
        self.CHANNELS = 1 # mono audio
        self.CHUNK = chunk_size # number of frames stored in each buffer

        self.audio = None
        self.stream = None

    def __enter__(self):
        assert self.stream is None, "This audio source is already inside a context manager"
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(
            input_device_index = self.device_index,
            format = self.format, rate = self.SAMPLE_RATE, channels = self.CHANNELS, frames_per_buffer = self.CHUNK,
            input = True, # stream is an input stream
        )
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.stream.stop_stream()
        self.stream.close()
        self.stream = None
        self.audio.terminate()

In [46]:
pyaudio.get_sample_size(pyaudio.paInt16)

2L

In [53]:
pyaudio.paInt16

8

In [48]:
print pyaudio.get_sample_size.__doc__


    Returns the size (in bytes) for the specified
    sample *format*.

    :param format: A |PaSampleFormat| constant.
    :raises ValueError: on invalid specified `format`.
    :rtype: integer
    


In [54]:
%%writefile recorder.py
import pyaudio

class AudioSource(object):
    def __init__(self):
        raise NotImplementedError("this is an abstract class")

    def __enter__(self):
        raise NotImplementedError("this is an abstract class")

    def __exit__(self, exc_type, exc_value, traceback):
        raise NotImplementedError("this is an abstract class")

class Recorder(AudioSource):
    
    def __init__(self, device_index = None, sample_rate = 16000, chunck_size =1024, bit_width = pyaudio.paInt16, channels = 1):
        assert device_index is None or isinstance(device_index, int), "`device_index` must be integer."
        if device_index is not None:
            audio = pyaudio.PyAudio()
            count = audio.get_device_count()
            audio.terminate()
            assert 0 <= device_index < count, "`device_index` out of range: {} out of {}".format(device_index, count)
        assert isinstance(sample_rate, int) and sample_rate > 0, "`sample_rate` must be positive."
        assert isinstance(chunck_size, int) and chunck_size > 0, "`chunck_size` must be positive."
        
        self.device_index = device_index
        self.sample_rate = sample_rate
        self.__chunck_size = chunck_size
        self.__bit_width = pyaudio.paInt16
        self.__sample_rate = sample_rate
        self.channels = channels
    
    @property
    def stream(self):
        audio_client = pyaudio.PyAudio()
        stream = audio_client.open()
        return stream
    
    @property
    def CHUNCK_SIZE(self):
        return self.__chunck_size
    
    @property
    def BIT_WIDTH(self):
        return self.__bit_width
    
    @property
    def SAMPLE_RATE(self):
        return self.__sample_rate
    
    @CHUNCK_SIZE.setter
    def CHUNCK_SIZE(self, value):
        if not self.__chunck_size is None:
            raise RuntimeError("Can not modify the `CHUNCK_SIZE`.")
        else:
            self.__chunck_size = value
    
    @BIT_WIDTH.setter
    def BIT_WIDTH(self, value):
        if not self.__bit_width is None:
            raise RuntimeError("Can not modify the `BIT_WIDTH`.")
        else:
            self.__bit_width = value
    
    @SAMPLE_RATE.setter
    def SAMPLE_RATE(self, value):
        if not self.__sample_rate is None:
            raise RuntimeError("Can not modify the `SAMPLE_RATE`.")
        else:
            self.__sample_rate = value    
        
    def listen(self):
        pass
    
    def plot(self):
        pass
    
    def play(self, start, end):
        pass

Writing recorder.py


In [6]:
pa = pyaudio.PyAudio()

In [9]:
print pa.get_device_count.__doc__


        Return the number of PortAudio Host APIs.

        :rtype: integer
        


In [10]:
pa.get_device_count()

3L

In [14]:
pa.get_default_host_api_info()

{'defaultInputDevice': 0L,
 'defaultOutputDevice': 1L,
 'deviceCount': 3L,
 'index': 0L,
 'name': u'Core Audio',
 'structVersion': 1L,
 'type': 5L}

In [25]:
pa.get_host_api_count()

1L

In [21]:
for i in range(3):
    print i
    print pa.get_device_info_by_host_api_device_index(0, 0)

0
{'defaultSampleRate': 44100.0, 'defaultLowOutputLatency': 0.01, 'defaultLowInputLatency': 0.002993197278911565, 'maxInputChannels': 2L, 'structVersion': 2L, 'hostApi': 0L, 'index': 0L, 'defaultHighOutputLatency': 0.1, 'maxOutputChannels': 0L, 'name': u'Built-in Microph', 'defaultHighInputLatency': 0.013151927437641724}
1
{'defaultSampleRate': 44100.0, 'defaultLowOutputLatency': 0.01, 'defaultLowInputLatency': 0.002993197278911565, 'maxInputChannels': 2L, 'structVersion': 2L, 'hostApi': 0L, 'index': 0L, 'defaultHighOutputLatency': 0.1, 'maxOutputChannels': 0L, 'name': u'Built-in Microph', 'defaultHighInputLatency': 0.013151927437641724}
2
{'defaultSampleRate': 44100.0, 'defaultLowOutputLatency': 0.01, 'defaultLowInputLatency': 0.002993197278911565, 'maxInputChannels': 2L, 'structVersion': 2L, 'hostApi': 0L, 'index': 0L, 'defaultHighOutputLatency': 0.1, 'maxOutputChannels': 0L, 'name': u'Built-in Microph', 'defaultHighInputLatency': 0.013151927437641724}


In [24]:
audio = pyaudio.PyAudio()
stream = audio.open(input_device_index = None, 
                    format = pyaudio.paInt16, rate = 16000, 
                    channels = 1, 
                    frames_per_buffer = 1024,
                    input = True)

In [26]:
audio.terminate()

In [44]:
## Auto detect audio device on host

import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
#for each audio device, determine if is an input or an output and add it to the appropriate list and dictionary
for i in range (0,numdevices):
        if p.get_device_info_by_host_api_device_index(0,i).get('maxInputChannels')>0:
                print "Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name')

        if p.get_device_info_by_host_api_device_index(0,i).get('maxOutputChannels')>0:
                print "Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name')
        
        if p.get_device_info_by_index(i)["maxInputChannels"] > 0:
            devinfo = p.get_device_info_by_index(i)
            
print "Selected device is ", devinfo.get('name')
if p.is_format_supported(44100.0,  # Sample rate
                         input_device=devinfo["index"],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
    print "Yay!"
p.terminate()

Input Device id  0  -  Built-in Microph
Output Device id  1  -  Built-in Output
Output Device id  2  -  DisplayPort
Selected device is  Built-in Microph
Yay!


In [37]:
devinfo = p.get_device_info_by_index(0)
devinfo

{'defaultHighInputLatency': 0.013151927437641724,
 'defaultHighOutputLatency': 0.1,
 'defaultLowInputLatency': 0.002993197278911565,
 'defaultLowOutputLatency': 0.01,
 'defaultSampleRate': 44100.0,
 'hostApi': 0L,
 'index': 0,
 'maxInputChannels': 2L,
 'maxOutputChannels': 0L,
 'name': u'Built-in Microph',
 'structVersion': 2L}

In [39]:
devinfo = p.get_device_info_by_index(1)
devinfo

{'defaultHighInputLatency': 0.1,
 'defaultHighOutputLatency': 0.031201814058956917,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.02104308390022676,
 'defaultSampleRate': 44100.0,
 'hostApi': 0L,
 'index': 1,
 'maxInputChannels': 0L,
 'maxOutputChannels': 2L,
 'name': u'Built-in Output',
 'structVersion': 2L}

In [42]:
devinfo = p.get_device_info_by_index(2)
devinfo

{'defaultHighInputLatency': 0.1,
 'defaultHighOutputLatency': 0.0118125,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.002479166666666667,
 'defaultSampleRate': 48000.0,
 'hostApi': 0L,
 'index': 2,
 'maxInputChannels': 0L,
 'maxOutputChannels': 2L,
 'name': u'DisplayPort',
 'structVersion': 2L}

In [33]:
devinfo["maxInputChannels"]

2L

In [34]:
p.is_format_supported(44100.0, input_device=devinfo["index"], input_channels = devinfo["maxInputChannels"], input_format=pyaudio.paInt16)

True